In [ ]:
import pandas as pd

# Use a forward slash or escape backslashes to avoid accidental escape sequences like '\f'
df = pd.read_csv("../Data/f1_dataset.csv")   # change to your actual path if different
df.head()


,race_id,race_name,race_date,race_year,winner_name,winner_team,full_text
0,1,2018 Australian Grand Prix,2018-03-25,2018,Lewis Hamilton,Mercedes,The 2018 Australian Grand Prix saw teams adapt...
1,2,2018 Bahrain Grand Prix,2018-04-08,2018,Lewis Hamilton,Mercedes,"At Sakhir, teams had to manage high track temp..."
2,3,2018 Chinese Grand Prix,2018-04-15,2018,Lewis Hamilton,Mercedes,Shanghai demanded a balanced package between d...
3,4,2018 Azerbaijan Grand Prix,2018-04-29,2018,Lewis Hamilton,Mercedes,Baku combined explosive straight-line sections...
4,5,2018 Spanish Grand Prix,2018-05-13,2018,Lewis Hamilton,Mercedes,Barcelona traditionally tests aero efficiency ...


In [19]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import joblib
le=LabelEncoder()
df["label"]=le.fit_transform(df["winner_name"])
joblib.dump(le, "models/label_encoder.pkl")
print(list(le.classes_))

df['winner_name'].value_counts()

['Charles Leclerc', 'George Russell', 'Lando Norris', 'Lewis Hamilton', 'Max Verstappen', 'Oscar Piastri', 'Pierre Gasly', 'Sergio Pérez', 'Valtteri Bottas']


winner_name
Lewis Hamilton     28
Max Verstappen     27
Lando Norris        9
Oscar Piastri       5
Sergio Pérez        5
Charles Leclerc     3
George Russell      2
Pierre Gasly        1
Valtteri Bottas     1
Name: count, dtype: int64

In [ ]:
# applying balancing by calculating median count
counts = df['label'].value_counts()
target = int(counts.median())
target

5

In [22]:
#applying balancing by setting all labels as target count
balanced_df = pd.DataFrame()

for label, count in counts.items():
    class_df = df[df['label'] == label]

    if count > target:
        # Undersample large classes
        class_df = class_df.sample(int(target), random_state=42)
    else:
        # Oversample small classes
        class_df = class_df.sample(int(target), replace=True, random_state=42)

    balanced_df = pd.concat([balanced_df, class_df], ignore_index=True)

# Shuffle final dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [24]:
balanced_df['winner_name'].value_counts()

winner_name
Pierre Gasly       5
Charles Leclerc    5
Valtteri Bottas    5
Lewis Hamilton     5
Lando Norris       5
Max Verstappen     5
Sergio Pérez       5
George Russell     5
Oscar Piastri      5
Name: count, dtype: int64

In [25]:
balanced_df.to_csv("../Data/balanced_f1_dataset.csv", index=False)